<a href="https://colab.research.google.com/github/Ash-Stat/An-Si-Hyun/blob/data/titanic_dataset_mean_median_bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [272]:
import pandas as pd
data=pd.read_csv('./train.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [273]:
# find a NaN
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [274]:
# drop featureless columns
data=data.drop(['PassengerId','Name'],axis=1)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [275]:
# find unique values
# 잘못 입력된 값이 있는지 확인
for i in data.columns:
  print(i,':', data[i].unique())

Survived : [0 1]
Pclass : [3 1 2]
Sex : ['male' 'female']
Age : [22.   38.   26.   35.     nan 54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.   28.    8.   19.   40.   66.   42.   21.
 18.    3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.
 16.   25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.
 14.5  70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.
 56.   50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.
 60.   10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.
  0.67 30.5   0.42 34.5  74.  ]
SibSp : [1 0 3 4 2 5 8]
Parch : [0 1 2 5 3 4 6]
Ticket : ['A/5 21171' 'PC 17599' 'STON/O2. 3101282' '113803' '373450' '330877'
 '17463' '349909' '347742' '237736' 'PP 9549' '113783' 'A/5. 2151'
 '347082' '350406' '248706' '382652' '244373' '345763' '2649' '239865'
 '248698' '330923' '113788' '347077' '2631' '19950' '330959' '349216'
 'PC 17601' 'PC 17569' '335677' 'C.A. 24579' 'PC 17604' '113789' '2677'


In [276]:
# 칼럼별 생존과의 연관성을 확인 : 오즈비
# 데이터셋 준비
import numpy as np
odds=pd.crosstab(index=data.Pclass, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
odds.loc[1]=odds.loc[1]/216
odds.loc[2]=odds.loc[2]/184
odds.loc[3]=odds.loc[3]/491
odds=odds[[1,0]]
odds

Survived,1,0
Pclass,,
1,0.629630,0.370370
2,0.472826,0.527174
3,0.242363,0.757637


In [277]:
#Pclass=1 vs 2
print('odds of Pclass 1 vs 2 :', 1/((80*87)/(136*97)))
print('odds of Pclass 1 vs 3 :', 1/((80*119)/(136*372)))
print('odds of Pclass 2 vs 3 :', 1/((97*119)/(87*372)))

odds of Pclass 1 vs 2 : 1.8954022988505748
odds of Pclass 1 vs 3 : 5.314285714285714
odds of Pclass 2 vs 3 : 2.803777180975483


In [278]:
# 아노바 분석
chi_data=pd.crosstab(index=data.Pclass, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
chi_data

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


In [279]:
import numpy as np
from scipy.stats import chi2_contingency
chi, p, dof, expected = chi2_contingency(chi_data)

print('Survived and Pclass')
print()
print(f"chi 스퀘어 값: {chi}",
      f"p-value (0.05): {p}",
      f"자유도 수: {dof}",
      f"기대값: \n{pd.DataFrame(expected)}",
      f"측정값: \n{chi_data}", sep = "\n" )
print('p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Pclass는 관계가 있다.')

Survived and Pclass

chi 스퀘어 값: 102.88898875696056
p-value (0.05): 4.549251711298793e-23
자유도 수: 2
기대값: 
            0           1
0  133.090909   82.909091
1  113.373737   70.626263
2  302.535354  188.464646
측정값: 
Survived    0    1
Pclass            
1          80  136
2          97   87
3         372  119
p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Pclass는 관계가 있다.


In [280]:
#성별과 생존
odds_sex_survived=pd.crosstab(index=data.Sex, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
odds_sex_survived=odds_sex_survived[[1,0]]
odds_sex_survived.loc['female']=odds_sex_survived.loc['female']/314
odds_sex_survived.loc['male']=odds_sex_survived.loc['male']/577
odds_sex_survived

Survived,1,0
Sex,,
female,0.742038,0.257962
male,0.188908,0.811092


In [281]:
print('odds of Sex female vs male :', (odds_sex_survived[1][0] * odds_sex_survived[0][1]) / (odds_sex_survived[1][1] * odds_sex_survived[0][0]))

odds of Sex female vs male : 12.3506625891947


In [282]:
# 아노바 분석
chi_Sex=pd.crosstab(index=data.Sex, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
chi_Sex=chi_Sex[[1,0]]
chi1, p1, dof1, expected1 = chi2_contingency(chi_Sex)

print('Survived and Sex')
print()
print(f"chi 스퀘어 값: {chi1}",
      f"p-value (0.05): {p1}",
      f"자유도 수: {dof1}",
      f"기대값: \n{pd.DataFrame(expected1)}",
      f"측정값: \n{chi_Sex}", sep = "\n" )
print('p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Sex는 관계가 있다.')

Survived and Sex

chi 스퀘어 값: 260.71702016732104
p-value (0.05): 1.1973570627755645e-58
자유도 수: 1
기대값: 
            0           1
0  120.525253  193.474747
1  221.474747  355.525253
측정값: 
Survived    1    0
Sex               
female    233   81
male      109  468
p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Sex는 관계가 있다.


In [283]:
# Age와 생존간의 관계를 확인하기 위해 로짓모형을 통해 분석한다.

import statsmodels.api as sm
age_survived=data[['Age','Survived']]
age_survived=age_survived.dropna()
logit = sm.Logit(age_survived['Survived'],age_survived['Age']) #로지스틱 회귀분석 시행
result = logit.fit()
result.summary2()

Optimization terminated successfully.
         Current function value: 0.672504
         Iterations 4


<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Logit
===============================================================
Model:              Logit            Pseudo R-squared: 0.004   
Dependent Variable: Survived         AIC:              962.3352
Date:               2021-01-02 10:22 BIC:              966.9061
No. Observations:   714              Log-Likelihood:   -480.17 
Df Model:           0                LL-Null:          -482.26 
Df Residuals:       713              LLR p-value:      nan     
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     4.0000                                     
-----------------------------------------------------------------
         Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
-----------------------------------------------------------------
Age     -0.0125     0.0024   -5.3284   0.0000   -0.0171   -0.0079
===============================================================

"""

In [284]:
sample=data[['Survived','Sex','Pclass','Age','SibSp','Parch']]
sample=sample.dropna()
sample

,Survived,Sex,Pclass,Age,SibSp,Parch
0,0,male,3,22.0,1,0
1,1,female,1,38.0,1,0
2,1,female,3,26.0,0,0
3,1,female,1,35.0,1,0
4,0,male,3,35.0,0,0
...,...,...,...,...,...,...
885,0,female,3,39.0,0,5
886,0,male,2,27.0,0,0
887,1,female,1,19.0,0,0
889,1,male,1,26.0,0,0


In [285]:
#베이즈 공식을 위한 자료정리
base=sample['Age'].groupby([sample['Survived'],sample['Sex'],sample['Pclass'],sample['SibSp'], sample['Parch']]).value_counts()
base=pd.DataFrame([base])
base=base.T
base.columns=['probability']
base

probability
Survived Sex    Pclass SibSp Parch Age              
0        female 1      0     0     50.0            1
                       1     2     2.0             1
                                   25.0            1
                2      0     0     24.0            1
                                   38.0            1
...                                              ...
1        male   3      1     1     4.0             1
                                   9.0             1
                                   20.0            1
                             2     1.0             1
                       4     2     3.0             1

[499 rows x 1 columns]

In [286]:
base.to_csv('./calculation.csv')

In [287]:
base1=pd.read_csv('./calculation.csv')
base1=base1['probability'].groupby([base1['Survived'], base1['Sex'], base1['Pclass'],base1['SibSp'],base1['Parch']]).sum()
base1=pd.DataFrame([base1])
base1=base1.T
base1

probability
Survived Sex    Pclass SibSp Parch             
0        female 1      0     0                1
                       1     2                2
                2      0     0                3
                       1     0                2
                             1                1
...                                         ...
1        male   3      0     2                1
                       1     0                3
                             1                4
                             2                1
                       4     2                1

[101 rows x 1 columns]

In [288]:
# 베이즈통계 구하기
for ind1,sum1 in zip(base1.index, base1['probability']):
  for ind2,val in zip(base.index, base['probability'].values):
    if ind1 == ind2[:5]:
      base['probability'].loc[ind2] = val/sum1
base

probability
Survived Sex    Pclass SibSp Parch Age              
0        female 1      0     0     50.0     1.000000
                       1     2     2.0      0.500000
                                   25.0     0.500000
                2      0     0     24.0     0.333333
                                   38.0     0.333333
...                                              ...
1        male   3      1     1     4.0      0.250000
                                   9.0      0.250000
                                   20.0     0.250000
                             2     1.0      1.000000
                       4     2     3.0      1.000000

[499 rows x 1 columns]

In [289]:
base.to_csv('./bayesian.csv')

In [290]:
# 예컨대 살아남지못하고, 여성이면서 Pclass가 1인 사람은 10세 이전 혹은 20대 혹은 50대일 확률이 각각 1/3이라는것이다.

#이 자료를 토대로 원래 데이터셋을 채워보면,
base=pd.read_csv('./bayesian.csv')
base

,Survived,Sex,Pclass,SibSp,Parch,Age,probability
0,0,female,1,0,0,50.0,1.000000
1,0,female,1,1,2,2.0,0.500000
2,0,female,1,1,2,25.0,0.500000
3,0,female,2,0,0,24.0,0.333333
4,0,female,2,0,0,38.0,0.333333
...,...,...,...,...,...,...,...
494,1,male,3,1,1,4.0,0.250000
495,1,male,3,1,1,9.0,0.250000
496,1,male,3,1,1,20.0,0.250000
497,1,male,3,1,2,1.0,1.000000


In [291]:
# base 데이터에서 더 높은 확률만을 갖는 데이터를 살아남게 한다.
# 곂치는 확률이 많으니 가중치를 구한다.
# 가중치 = probability * age
for Sur,Se,Pc,Pro,Sib,Par,inin in zip(base['Survived'], base['Sex'], base['Pclass'], base['probability'], base['SibSp'],base['Parch'],base.index):
  for Sur1,Se1,Pc1,Pro1,Sib1,Par1, inin1 in zip(base['Survived'], base['Sex'], base['Pclass'], base['probability'], base['SibSp'],base['Parch'], base.index):
    if Sur == Sur1 and Se == Se1 and Pc == Pc1 and Sib == Sib1 and Par == Par1 and Pro > Pro1:
      base['probability'].loc[inin1]=np.nan
base=base.dropna()
base['weighted_age']=base['Age']*base['probability']
base=base['weighted_age'].groupby([base['Survived'],base['Sex'],base['Pclass'],base['SibSp'],base['Parch']]).mean()
base=pd.DataFrame([base])
base=base.T
base.to_csv('./weighted_age.csv')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [292]:
base=pd.read_csv('./weighted_age.csv')
base

,Survived,Sex,Pclass,SibSp,Parch,weighted_age
0,0,female,1,0,0,50.000000
1,0,female,1,1,2,6.750000
2,0,female,2,0,0,13.222222
3,0,female,2,1,0,17.750000
4,0,female,2,1,1,26.000000
...,...,...,...,...,...,...
96,1,male,3,0,2,9.000000
97,1,male,3,1,0,6.333333
98,1,male,3,1,1,2.250000
99,1,male,3,1,2,1.000000


In [293]:
data_null=data[data['Age'].isnull() == True]
data_null
for Sur,Se,Pc,Sib,Par,ag,inin in zip(base['Survived'], base['Sex'], base['Pclass'],base['SibSp'],base['Parch'],base['weighted_age'],base.index):
  for Sur1,Se1,Pc1,Sib1,Par1,agag, inin1 in zip(data_null['Survived'], data_null['Sex'], data_null['Pclass'],data_null['SibSp'],data_null['Parch'],data_null['Age'], data_null.index):
    if Sur == Sur1 and Se == Se1 and Pc == Pc1 and Sib == Sib1 and Par == Par1:
      data_null['Age'].loc[inin1]=ag
data_null

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,0,3,male,1.602484,0,0,330877,8.4583,NaN,Q
17,1,2,male,7.520000,0,0,244373,13.0000,NaN,S
19,1,3,female,4.190476,0,0,2649,7.2250,NaN,C
26,0,3,male,1.602484,0,0,2631,7.2250,NaN,C
28,1,3,female,4.190476,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...
859,0,3,male,1.602484,0,0,2629,7.2292,NaN,C
863,0,3,female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,0,3,male,1.602484,0,0,345777,9.5000,NaN,S
878,0,3,male,1.602484,0,0,349217,7.8958,NaN,S


In [294]:
for wa,inde in zip(data_null['Age'], data_null.index):
  data['Age'].loc[inde] = wa
data    

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [295]:
#못채운 결측치는 총 10개
data_remain=data[data['Age'].isnull() == True]
data_remain

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
159,0,3,male,NaN,8,2,CA. 2343,69.55,NaN,S
180,0,3,female,NaN,8,2,CA. 2343,69.55,NaN,S
201,0,3,male,NaN,8,2,CA. 2343,69.55,NaN,S
301,1,3,male,NaN,2,0,367226,23.25,NaN,Q
324,0,3,male,NaN,8,2,CA. 2343,69.55,NaN,S
330,1,3,female,NaN,2,0,367226,23.25,NaN,Q
792,0,3,female,NaN,8,2,CA. 2343,69.55,NaN,S
846,0,3,male,NaN,8,2,CA. 2343,69.55,NaN,S
863,0,3,female,NaN,8,2,CA. 2343,69.55,NaN,S
888,0,3,female,NaN,1,2,W./C. 6607,23.45,NaN,S


In [296]:
#베이즈 공식으로 채운 결측값을 기반으로 다시 관계를 확인해 본다.
# Age와 생존간의 관계를 확인하기 위해 로짓모형을 통해 분석한다.
age_survived2=data[['Age','Survived']]
age_survived2=age_survived2.dropna()
logit2 = sm.Logit(age_survived2['Survived'],age_survived2['Age']) #로지스틱 회귀분석 시행
result2 = logit2.fit()
result2.summary2()

Optimization terminated successfully.
         Current function value: 0.675851
         Iterations 4


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
================================================================
Model:              Logit            Pseudo R-squared: -0.013   
Dependent Variable: Survived         AIC:              1192.8494
Date:               2021-01-02 10:22 BIC:              1197.6304
No. Observations:   881              Log-Likelihood:   -595.42  
Df Model:           0                LL-Null:          -587.53  
Df Residuals:       880              LLR p-value:      nan      
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     4.0000                                      
------------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
------------------------------------------------------------------
Age      -0.0127     0.0023   -5.4151   0.0000   -0.0173   -0.0081
================================================================

"""

In [297]:
fare_survived=data[['Fare','Survived']]
fare_survived=fare_survived.dropna()
logit3 = sm.Logit(fare_survived['Survived'],fare_survived['Fare']) #로지스틱 회귀분석 시행
result3 = logit3.fit()
result3.summary2()

#유의하다.(상관성이 있다.)

Optimization terminated successfully.
         Current function value: 0.689550
         Iterations 4


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
================================================================
Model:              Logit            Pseudo R-squared: -0.035   
Dependent Variable: Survived         AIC:              1230.7774
Date:               2021-01-02 10:22 BIC:              1235.5697
No. Observations:   891              Log-Likelihood:   -614.39  
Df Model:           0                LL-Null:          -593.33  
Df Residuals:       890              LLR p-value:      nan      
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     4.0000                                      
-------------------------------------------------------------------
         Coef.     Std.Err.      z       P>|z|     [0.025    0.975]
-------------------------------------------------------------------
Fare     0.0030      0.0012    2.4342    0.0149    0.0006    0.0054
================================================================

"""

In [298]:
#Embakred랑 survived 오즈비, 아노바 분석

em_survived=data[['Embarked','Survived']]
em_survived=em_survived.dropna()
em_survived

,Embarked,Survived
0,S,0
1,C,1
2,S,1
3,S,1
4,S,0
...,...,...
886,S,0
887,S,1
888,S,0
889,C,1


In [299]:
# 오즈비 함수
def odds(dat,init,m):
  for n in range(init,m):
    print('odds %ds vs %ds:' % (init,n+1),(dat[1].iloc[init] * dat[0].iloc[n+1]) / (dat[1].iloc[n+1] * dat[0].iloc[init]))

In [300]:
em_odds=pd.crosstab(index=data.Embarked, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
em_odds.iloc[0]=em_odds.iloc[0]/168
em_odds.iloc[1]=em_odds.iloc[1]/77
em_odds.iloc[2]=em_odds.iloc[2]/644
em_odds=em_odds[[1,0]]
em_odds

Survived,1,0
Embarked,,
C,0.553571,0.446429
Q,0.389610,0.610390
S,0.336957,0.663043


In [301]:
odds(em_odds,0,2)

odds 0s vs 1s: 1.9426666666666668
odds 0s vs 2s: 2.44


In [302]:
odds(em_odds,1,2)

odds 1s vs 2s: 1.2560054907343856


In [303]:
# 오즈비 아노바 분석
em_anova=pd.crosstab(index=data.Embarked, columns=data.Survived,values=data.Survived, aggfunc=pd.value_counts )
em_anova=em_anova[[1,0]]
chi4, p4, dof4, expected4 = chi2_contingency(em_anova)

print('Survived and Embarked')
print()
print(f"chi 스퀘어 값: {chi4}",
      f"p-value (0.05): {p4}",
      f"자유도 수: {dof4}",
      f"기대값: \n{pd.DataFrame(expected4)}",
      f"측정값: \n{em_anova}", sep = "\n" )
print('p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Embarked는 관계가 있다.')

Survived and Embarked

chi 스퀘어 값: 26.48914983923762
p-value (0.05): 1.769922284120912e-06
자유도 수: 2
기대값: 
            0           1
0   64.251969  103.748031
1   29.448819   47.551181
2  246.299213  397.700787
측정값: 
Survived    1    0
Embarked          
C          93   75
Q          30   47
S         217  427
p-value가 0.05 이하 이므로, 귀무가설을 기각한다. 즉, 생존과 Embarked는 관계가 있다.


In [304]:
#Sib 와 Survived 로짓모형분석
sib_survived=data[['SibSp','Survived']]
sib_survived=sib_survived.dropna()
logit4 = sm.Logit(sib_survived['Survived'],sib_survived['SibSp']) #로지스틱 회귀분석 시행
result4 = logit4.fit()
result4.summary2()


Optimization terminated successfully.
         Current function value: 0.684112
         Iterations 5


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
================================================================
Model:              Logit            Pseudo R-squared: -0.027   
Dependent Variable: Survived         AIC:              1221.0882
Date:               2021-01-02 10:22 BIC:              1225.8806
No. Observations:   891              Log-Likelihood:   -609.54  
Df Model:           0                LL-Null:          -593.33  
Df Residuals:       890              LLR p-value:      nan      
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     5.0000                                      
------------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
------------------------------------------------------------------
SibSp    -0.2401     0.0646   -3.7154   0.0002   -0.3668   -0.1134
================================================================

"""

In [305]:
#Parch 와 Survived 로짓모형분석
par_survived=data[['Parch','Survived']]
par_survived=par_survived.dropna()
logit5 = sm.Logit(par_survived['Survived'],par_survived['Parch']) #로지스틱 회귀분석 시행
result5 = logit5.fit()
result5.summary2()

# 즉, Parch와 Survived와의 상관성이 있다고 할 수는 없다.

Optimization terminated successfully.
         Current function value: 0.692763
         Iterations 3


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
================================================================
Model:              Logit            Pseudo R-squared: -0.040   
Dependent Variable: Survived         AIC:              1236.5035
Date:               2021-01-02 10:22 BIC:              1241.2959
No. Observations:   891              Log-Likelihood:   -617.25  
Df Model:           0                LL-Null:          -593.33  
Df Residuals:       890              LLR p-value:      nan      
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     3.0000                                      
------------------------------------------------------------------
           Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
------------------------------------------------------------------
Parch     -0.0624     0.0755   -0.8255   0.4091   -0.2104   0.0857
================================================================

"""

In [306]:
# 데이터에서 제거할것 : Parch, Ticket, Cabin

#Cabin 을 제거해야하는 이유는, 결측값이 원본 데이터에비해서 너무 많기 때문에 무리해서 채우는것은 바람직하지 않으므로, 제거한다.

data=data.drop(['Parch','Ticket','Cabin'],axis=1)
data

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,male,22.0,1,7.2500,S
1,1,1,female,38.0,1,71.2833,C
2,1,3,female,26.0,0,7.9250,S
3,1,1,female,35.0,1,53.1000,S
4,0,3,male,35.0,0,8.0500,S
...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,13.0000,S
887,1,1,female,19.0,0,30.0000,S
888,0,3,female,NaN,1,23.4500,S
889,1,1,male,26.0,0,30.0000,C


In [307]:
#나머지 결측값 제거

data=data.dropna()
data=data.reset_index()
data

,index,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,0,3,male,22.0,1,7.2500,S
1,1,1,1,female,38.0,1,71.2833,C
2,2,1,3,female,26.0,0,7.9250,S
3,3,1,1,female,35.0,1,53.1000,S
4,4,0,3,male,35.0,0,8.0500,S
...,...,...,...,...,...,...,...,...
874,885,0,3,female,39.0,0,29.1250,Q
875,886,0,2,male,27.0,0,13.0000,S
876,887,1,1,female,19.0,0,30.0000,S
877,889,1,1,male,26.0,0,30.0000,C


In [308]:
# 범주형 자료 숫자로 바꿔주기
from sklearn.preprocessing import OrdinalEncoder
encoder=OrdinalEncoder()
categories=encoder.fit_transform(data[['Sex','Embarked']])
categories=pd.DataFrame(categories)
categories

,0,1
0,1.0,2.0
1,0.0,0.0
2,0.0,2.0
3,0.0,2.0
4,1.0,2.0
...,...,...
874,0.0,1.0
875,1.0,2.0
876,0.0,2.0
877,1.0,0.0


In [309]:
data['Sex']=categories[0]
data['Embarked']=categories[1]
data

,index,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,0,3,1.0,22.0,1,7.2500,2.0
1,1,1,1,0.0,38.0,1,71.2833,0.0
2,2,1,3,0.0,26.0,0,7.9250,2.0
3,3,1,1,0.0,35.0,1,53.1000,2.0
4,4,0,3,1.0,35.0,0,8.0500,2.0
...,...,...,...,...,...,...,...,...
874,885,0,3,0.0,39.0,0,29.1250,1.0
875,886,0,2,1.0,27.0,0,13.0000,2.0
876,887,1,1,0.0,19.0,0,30.0000,2.0
877,889,1,1,1.0,26.0,0,30.0000,0.0


In [310]:
data=data.drop(['index'],axis=1)
data

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,1.0,22.0,1,7.2500,2.0
1,1,1,0.0,38.0,1,71.2833,0.0
2,1,3,0.0,26.0,0,7.9250,2.0
3,1,1,0.0,35.0,1,53.1000,2.0
4,0,3,1.0,35.0,0,8.0500,2.0
...,...,...,...,...,...,...,...
874,0,3,0.0,39.0,0,29.1250,1.0
875,0,2,1.0,27.0,0,13.0000,2.0
876,1,1,0.0,19.0,0,30.0000,2.0
877,1,1,1.0,26.0,0,30.0000,0.0


In [311]:
data.to_csv('./finaldata.csv',encoding='utf-8-sig',index=False)

In [312]:
# 트레인 테스트데이터 나누어 보기.
from sklearn.model_selection import train_test_split

train_set,test_set=train_test_split(data, test_size=0.3, random_state=42)

In [313]:
train_y=train_set['Survived']
train_x=train_set.drop(['Survived'],axis=1)
test_y=test_set['Survived']
test_x=test_set.drop(['Survived'],axis=1)

In [314]:
#랜덤포레스트 적용
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
forest=RandomForestClassifier()
forest.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [315]:
forest.predict(test_x)
accuracy_score(test_y,forest.predict(test_x))

0.803030303030303

In [316]:
#교차검증 해보기
from sklearn.model_selection import cross_val_score
score=cross_val_score(forest,train_x,train_y,scoring='accuracy',cv=5)
score.mean()

#교차검증 결과값으로 보아 과적합이 되어있지는 않다.

0.8048780487804879

# 하이퍼 파라미터 설계

In [317]:
#그리드탐색

from sklearn.model_selection import GridSearchCV
fore=RandomForestClassifier()
parameter=[
           {'n_estimators' : [100], 'max_features' : [1]}, #3*4=12의 경우의수 탐색
           {'bootstrap' : [False], 'n_estimators' : [100], 'max_features' : [1]} #3*3=9개의 경우의수 탐색
] #refit=True로 하면 최적의 파라미터를 찾은후 모델을 수정시킨다.
grid=GridSearchCV(fore,parameter,cv=5,scoring='accuracy',return_train_score=True)
grid.fit(train_x,train_y)
#모델넣고, 파라미터설정한거 넣고, 폴드수 넣고, 점수방식 넣고, 훈련점수표현할지말지넣고

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [318]:
print('최적의 파라미터 :',grid.best_params_)
print('최적의 추정기 :',grid.best_estimator_)

최적의 파라미터 : {'max_features': 1, 'n_estimators': 100}
최적의 추정기 : RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=1,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [319]:
result=grid.cv_results_
for score, params in zip(result['mean_test_score'], result['params']):
  print('점수 :',score,' ','파라미터 :',params)

점수 : 0.8065040650406503   파라미터 : {'max_features': 1, 'n_estimators': 100}
점수 : 0.7983739837398374   파라미터 : {'bootstrap': False, 'max_features': 1, 'n_estimators': 100}


In [320]:
grid.best_score_

0.8065040650406503

#결측값을 평균으로 대체했을때는??

In [321]:
import pandas as pd
data1=pd.read_csv('./train.csv')
data1

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [322]:
# drop featureless columns
data1=data1.drop(['PassengerId','Name','Parch','Ticket','Cabin'],axis=1)
data1

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,male,22.0,1,7.2500,S
1,1,1,female,38.0,1,71.2833,C
2,1,3,female,26.0,0,7.9250,S
3,1,1,female,35.0,1,53.1000,S
4,0,3,male,35.0,0,8.0500,S
...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,13.0000,S
887,1,1,female,19.0,0,30.0000,S
888,0,3,female,NaN,1,23.4500,S
889,1,1,male,26.0,0,30.0000,C


In [323]:
data1['Age']=data1['Age'].fillna(data1['Age'].mean())
data1

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,male,22.000000,1,7.2500,S
1,1,1,female,38.000000,1,71.2833,C
2,1,3,female,26.000000,0,7.9250,S
3,1,1,female,35.000000,1,53.1000,S
4,0,3,male,35.000000,0,8.0500,S
...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,13.0000,S
887,1,1,female,19.000000,0,30.0000,S
888,0,3,female,29.699118,1,23.4500,S
889,1,1,male,26.000000,0,30.0000,C


In [324]:
data1=data1.dropna()
data1=data1.reset_index()
data1=data1.drop(['index'],axis=1)
data1

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,male,22.000000,1,7.2500,S
1,1,1,female,38.000000,1,71.2833,C
2,1,3,female,26.000000,0,7.9250,S
3,1,1,female,35.000000,1,53.1000,S
4,0,3,male,35.000000,0,8.0500,S
...,...,...,...,...,...,...,...
884,0,2,male,27.000000,0,13.0000,S
885,1,1,female,19.000000,0,30.0000,S
886,0,3,female,29.699118,1,23.4500,S
887,1,1,male,26.000000,0,30.0000,C


In [325]:
encoder1=OrdinalEncoder()
categories1=encoder1.fit_transform(data1[['Sex','Embarked']])
categories1=pd.DataFrame(categories1)
categories1

,0,1
0,1.0,2.0
1,0.0,0.0
2,0.0,2.0
3,0.0,2.0
4,1.0,2.0
...,...,...
884,1.0,2.0
885,0.0,2.0
886,0.0,2.0
887,1.0,0.0


In [326]:
data1['Sex']=categories1[0]
data1['Embarked']=categories1[1]
data1

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,1.0,22.000000,1,7.2500,2.0
1,1,1,0.0,38.000000,1,71.2833,0.0
2,1,3,0.0,26.000000,0,7.9250,2.0
3,1,1,0.0,35.000000,1,53.1000,2.0
4,0,3,1.0,35.000000,0,8.0500,2.0
...,...,...,...,...,...,...,...
884,0,2,1.0,27.000000,0,13.0000,2.0
885,1,1,0.0,19.000000,0,30.0000,2.0
886,0,3,0.0,29.699118,1,23.4500,2.0
887,1,1,1.0,26.000000,0,30.0000,0.0


In [327]:
data1.to_csv('./meandata.csv',encoding='utf-8-sig',index=False)

# 중간값으로 대체했을때

In [328]:
import pandas as pd
data2=pd.read_csv('./train.csv')
# drop featureless columns
data2=data2.drop(['PassengerId','Name','Parch','Ticket','Cabin'],axis=1)
data2['Age']=data2['Age'].fillna(data2['Age'].median())
data2=data2.dropna()
data2=data2.reset_index()
data2=data2.drop(['index'],axis=1)
encoder2=OrdinalEncoder()
categories2=encoder2.fit_transform(data2[['Sex','Embarked']])
categories2=pd.DataFrame(categories2)
categories2
data2['Sex']=categories2[0]
data2['Embarked']=categories2[1]
data2.to_csv('./mediandata.csv',encoding='utf-8-sig',index=False)

In [329]:
data2

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,1.0,22.0,1,7.2500,2.0
1,1,1,0.0,38.0,1,71.2833,0.0
2,1,3,0.0,26.0,0,7.9250,2.0
3,1,1,0.0,35.0,1,53.1000,2.0
4,0,3,1.0,35.0,0,8.0500,2.0
...,...,...,...,...,...,...,...
884,0,2,1.0,27.0,0,13.0000,2.0
885,1,1,0.0,19.0,0,30.0000,2.0
886,0,3,0.0,28.0,1,23.4500,2.0
887,1,1,1.0,26.0,0,30.0000,0.0


# 결측값 싹 지워버리기

In [330]:
import pandas as pd
data3=pd.read_csv('./train.csv')
# drop featureless columns
data3=data3.drop(['PassengerId','Name','Parch','Ticket','Cabin'],axis=1)
data3=data3.dropna()
data3=data3.reset_index()
data3=data3.drop(['index'],axis=1)
encoder3=OrdinalEncoder()
categories3=encoder3.fit_transform(data2[['Sex','Embarked']])
categories3=pd.DataFrame(categories3)
categories3
data3['Sex']=categories3[0]
data3['Embarked']=categories3[1]
data3.to_csv('./dropdata.csv',encoding='utf-8-sig',index=False)
data3

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,1.0,22.0,1,7.2500,2.0
1,1,1,0.0,38.0,1,71.2833,0.0
2,1,3,0.0,26.0,0,7.9250,2.0
3,1,1,0.0,35.0,1,53.1000,2.0
4,0,3,1.0,35.0,0,8.0500,2.0
...,...,...,...,...,...,...,...
707,0,3,0.0,39.0,0,29.1250,2.0
708,0,2,1.0,27.0,0,13.0000,0.0
709,1,1,0.0,19.0,0,30.0000,0.0
710,1,1,1.0,26.0,0,30.0000,2.0


# 테스트시작

In [331]:
test=pd.read_csv('./test.csv')
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

# 베이즈통계로 결측값 채우기

In [332]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [333]:
data_null=test[test['Age'].isnull() == True]
data_null
for Se,Pc,Sib,Par,ag,inin in zip(base['Sex'], base['Pclass'],base['SibSp'],base['Parch'],base['weighted_age'],base.index):
  for Se1,Pc1,Sib1,Par1,agag, inin1 in zip(data_null['Sex'], data_null['Pclass'],data_null['SibSp'],data_null['Parch'],data_null['Age'], data_null.index):
    if Se == Se1 and Pc == Pc1 and Sib == Sib1 and Par == Par1:
      data_null['Age'].loc[inin1]=ag
data_null

for wa,inde in zip(data_null['Age'], data_null.index):
  test['Age'].loc[inde] = wa
test

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.500000,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.000000,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.000000,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.000000,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.000000,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,6.153846,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,6.153846,0,0,359309,8.0500,NaN,S


In [334]:
# 나머지는 평균값으로 대체 한다.

test=test.drop(['PassengerId','Name','Ticket','Cabin','Parch'],axis=1)
test

,Pclass,Sex,Age,SibSp,Fare,Embarked
0,3,male,34.500000,0,7.8292,Q
1,3,female,47.000000,1,7.0000,S
2,2,male,62.000000,0,9.6875,Q
3,3,male,27.000000,0,8.6625,S
4,3,female,22.000000,1,12.2875,S
...,...,...,...,...,...,...
413,3,male,6.153846,0,8.0500,S
414,1,female,39.000000,0,108.9000,C
415,3,male,38.500000,0,7.2500,S
416,3,male,6.153846,0,8.0500,S


In [335]:
test['Age']=test['Age'].fillna(test['Age'].mean())
test

,Pclass,Sex,Age,SibSp,Fare,Embarked
0,3,male,34.500000,0,7.8292,Q
1,3,female,47.000000,1,7.0000,S
2,2,male,62.000000,0,9.6875,Q
3,3,male,27.000000,0,8.6625,S
4,3,female,22.000000,1,12.2875,S
...,...,...,...,...,...,...
413,3,male,6.153846,0,8.0500,S
414,1,female,39.000000,0,108.9000,C
415,3,male,38.500000,0,7.2500,S
416,3,male,6.153846,0,8.0500,S


In [336]:
test['Fare']=test['Fare'].fillna(test['Fare'].mean())
test

,Pclass,Sex,Age,SibSp,Fare,Embarked
0,3,male,34.500000,0,7.8292,Q
1,3,female,47.000000,1,7.0000,S
2,2,male,62.000000,0,9.6875,Q
3,3,male,27.000000,0,8.6625,S
4,3,female,22.000000,1,12.2875,S
...,...,...,...,...,...,...
413,3,male,6.153846,0,8.0500,S
414,1,female,39.000000,0,108.9000,C
415,3,male,38.500000,0,7.2500,S
416,3,male,6.153846,0,8.0500,S


In [337]:
encoder2=OrdinalEncoder()
categories2=encoder2.fit_transform(test[['Sex','Embarked']])
categories2=pd.DataFrame(categories2)
categories2
test['Sex']=categories2[0]
test['Embarked']=categories2[1]
test.to_csv('./test_base.csv',encoding='utf-8-sig',index=False)

In [338]:
result=pd.read_csv('./survive_result.csv')
result=result.T
result.columns=['Survived']
result

,Survived
0,0
1,0
2,0
3,1
4,0
...,...
413,1
414,1
415,0
416,1


In [339]:
raw=pd.read_csv('./test.csv')
raw

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [340]:
result.index=raw['PassengerId']
result

,Survived
PassengerId,
892,0
893,0
894,0
895,1
896,0
...,...
1305,1
1306,1
1307,0


In [341]:
result=result.reset_index()
result

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,1
414,1306,1
415,1307,0
416,1308,1


In [342]:
result.to_csv('./submission.csv',encoding='utf-8-sig',index=False)

#평균 대체

In [343]:
test_mean=pd.read_csv('./test.csv')
test_mean=test_mean.drop(['PassengerId','Name','Ticket','Cabin','Parch'],axis=1)
test_mean['Age']=test_mean['Age'].fillna(test['Age'].mean())
test_mean['Fare']=test_mean['Fare'].fillna(test['Fare'].mean())
encoder2=OrdinalEncoder()
categories2=encoder2.fit_transform(test_mean[['Sex','Embarked']])
categories2=pd.DataFrame(categories2)
categories2
test_mean['Sex']=categories2[0]
test_mean['Embarked']=categories2[1]
test_mean.to_csv('./test_mean.csv',encoding='utf-8-sig',index=False)

In [344]:
result=pd.read_csv('./survive_result_mean.csv')
result=result.T
result.columns=['Survived']

raw=pd.read_csv('./test.csv')
raw

result.index=raw['PassengerId']
result

result=result.reset_index()
result

result.to_csv('./submission_mean.csv',encoding='utf-8-sig',index=False)

# 중간값 대체

In [345]:
test_median=pd.read_csv('./test.csv')
test_median=test_median.drop(['PassengerId','Name','Ticket','Cabin','Parch'],axis=1)
test_median['Age']=test_median['Age'].fillna(test['Age'].median())
test_median['Fare']=test_median['Fare'].fillna(test['Fare'].median())
encoder2=OrdinalEncoder()
categories2=encoder2.fit_transform(test_median[['Sex','Embarked']])
categories2=pd.DataFrame(categories2)
categories2
test_median['Sex']=categories2[0]
test_median['Embarked']=categories2[1]
test_median.to_csv('./test_median.csv',encoding='utf-8-sig',index=False)

In [346]:
result=pd.read_csv('./survive_result_median.csv')
result=result.T
result.columns=['Survived']

raw=pd.read_csv('./test.csv')
raw

result.index=raw['PassengerId']
result

result=result.reset_index()
result

result.to_csv('./submission_median.csv',encoding='utf-8-sig',index=False)